<a href="https://www.kaggle.com/code/abhinandan0506/vgg19-lstm-svm-model?scriptVersionId=180207067" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
import numpy as np
from keras.models import Model
from keras.layers import Input, LSTM, Reshape, BatchNormalization, Flatten, Dense
from keras.applications.vgg19 import VGG19
from keras.optimizers import Adam
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import tensorflow as tf
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping
from math import floor

# Defining directories

In [ ]:
train_dir = '/kaggle/input/brain-tumor-mri-dataset/Training/'
test_dir = '/kaggle/input/brain-tumor-mri-dataset/Testing/'

# Hyperparameters

In [ ]:
EPOCHS = 50
INIT_LR = 1e-3
decay_rate = 0.95
decay_step = 1
IMAGE_SIZE = [224, 224]

In [ ]:
def Combined_model():
    # Input layer
    input_layer = Input(shape=(224, 224, 3))

    # Base VGG19 model as a feature extractor
    baseModel = VGG19(weights='imagenet', include_top=False, input_tensor=input_layer)

    # Freeze the layers of the VGG19 model
    for layer in baseModel.layers:
        layer.trainable = False

    x = baseModel.output

    # LSTM layer
    x = Reshape((49, 512))(x)
    x = LSTM(512, activation="relu", return_sequences=True, trainable=False)(x)
    x = BatchNormalization()(x)

    # Flatten output from LSTM
    x = Flatten()(x)

    # Output layer
    output = Dense(4, activation='softmax')(x)

    # Define the model
    model = Model(inputs=input_layer, outputs=output)

    opt = Adam(learning_rate=INIT_LR)
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=["accuracy"])
    
    return model

# Creating the model

In [ ]:
model = Combined_model()

In [ ]:
model.summary()

# Load and preprocess data

In [ ]:
train_data = tf.keras.preprocessing.image_dataset_from_directory(train_dir,
                                                                 label_mode="categorical",
                                                                 batch_size=32,
                                                                 image_size=(224, 224))

test_data = tf.keras.preprocessing.image_dataset_from_directory(test_dir,
                                                                label_mode="categorical",
                                                                image_size=(224, 224),
                                                                shuffle=False)

# Data augmentation

In [ ]:
def augment_image(image, label):
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    image = tf.image.random_brightness(image, max_delta=0.1)
    image = tf.image.random_contrast(image, lower=0.8, upper=1.2)
    image = tf.image.random_saturation(image, lower=0.8, upper=1.2)
    image = tf.image.random_hue(image, max_delta=0.1)
    return image, label

In [ ]:
train_dataset_augmented = train_data.map(augment_image)

In [ ]:
checkpoint = ModelCheckpoint(filepath='best_model.weights.h5', 
                             monitor='val_accuracy', 
                             mode='max', 
                             verbose=1, 
                             save_best_only=True, 
                             save_weights_only=True)
earlystop = EarlyStopping(monitor='accuracy', 
                          min_delta=0, 
                          patience=15, 
                          verbose=1, 
                          mode='max')
lr_scheduler = LearningRateScheduler(lambda epoch: INIT_LR * pow(decay_rate, floor(epoch / decay_step)))

# Training the model

In [ ]:
history = model.fit(train_dataset_augmented,
                    epochs=EPOCHS,
                    steps_per_epoch=len(train_dataset_augmented),
                    validation_data=test_data,
                    callbacks=[checkpoint, earlystop, lr_scheduler])

# Extracting features

In [ ]:
def extract_features(data_generator, model):
    features = []
    labels = []
    for images, batch_labels in data_generator:
        batch_features = model.predict(images)
        features.append(batch_features)
        labels.append(batch_labels.numpy())  # Convert labels to numpy arrays

    # Concatenate all the features and labels
    features = np.concatenate(features, axis=0)
    labels = np.concatenate(labels, axis=0)
    
    return features, labels

In [ ]:
train_features, train_labels = extract_features(train_data, model)
test_features, test_labels = extract_features(test_data, model)

In [ ]:
train_labels = np.argmax(train_labels, axis=1)
test_labels = np.argmax(test_labels, axis=1)

# Training SVM on extracted features

In [ ]:
scaler = StandardScaler()
num_samples, num_features = train_features.shape
pca_components = min(num_samples, num_features, 64)  # Adjust the number of PCA components as needed
pca = PCA(n_components=pca_components)
svm_model = SVC(kernel="linear", C=1.0, probability=True)

In [ ]:
train_features_scaled = scaler.fit_transform(train_features)
train_features_pca = pca.fit_transform(train_features_scaled)

In [ ]:
svm_model.fit(train_features_pca, train_labels)

In [ ]:
test_features_scaled = scaler.transform(test_features)
test_features_pca = pca.transform(test_features_scaled)
accuracy = svm_model.score(test_features_pca, test_labels)
print(f'SVM Accuracy: {accuracy}')

# Classification Report

In [ ]:
from sklearn.metrics import classification_report

test_predictions = svm_model.predict(test_features_pca)

report = classification_report(test_labels, test_predictions, target_names=test_data.class_names)

print(report)

# Confusion Matrix

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

# Predict the labels of the test set using the trained SVM
test_predictions = svm_model.predict(test_features_pca)

# Generate the confusion matrix
cm = confusion_matrix(test_labels, test_predictions)

# Plot the confusion matrix
plt.figure(figsize=(10, 7))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=test_data.class_names, yticklabels=test_data.class_names)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 8))
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# Loss Graph

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 8))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()